<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto2/Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparando o ambiente

In [ ]:
import pandas as pd
import numpy as np
import re
import gc
import warnings
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


##Carregando o dataset e processando os dados
No projeto 1, foi adicionado o dump dos dados para que possamos utiliza-los aqui. O objetivo desse projeto é predizer a popularidade de uma música através de suas caracteristicas informadas no subset.

In [ ]:
df = pd.read_csv('project1_output.csv')

Observamos que o dataset é grande e assim decidimos utilizar um subset  com o objetivo de diminuir o tempo de experimentação

In [ ]:

df = df[:1000]
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.dtypes

year                   int64
acousticness         float64
artists_data          object
danceability         float64
duration_ms            int64
energy               float64
explicit               int64
id                    object
instrumentalness     float64
loudness             float64
name                  object
popularity           float64
release_date          object
speechiness          float64
tempo                float64
first_artist          object
artists_by_artist     object
first_genre           object
dtype: object

Tirando colunas desnecessárias, como `artist_data, id, name, release_date, first_artist, artists_by_artist` e `first_genre,` pois só queremos trabalhar com valores numéricos




In [ ]:
df = df.select_dtypes(exclude=['object'])

In [ ]:
df.columns

Index(['year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'loudness', 'popularity', 'speechiness',
       'tempo'],
      dtype='object')



###Escolher coluna para predição

In [ ]:
target_col = df['popularity']

In [ ]:
target_col

0      0.04
1      0.02
2      0.04
3      0.00
4      0.01
       ... 
995    0.00
996    0.00
997    0.00
998    0.00
999    0.00
Name: popularity, Length: 1000, dtype: float64

In [ ]:
df = df.drop(columns=['popularity'])

##Separando os dados em teste e predição
Para realizacão do treinamento, teste e vaidacão, iremos separa o subset em: 

*   3/5 dos dados para treinamento
*   1/5 dos dados para teste 
*   1/5 dos dados para validacão

Seguindo assim, uma base 60/20/20



In [ ]:
def get_x_data():
    # input 
    train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    
    return train, val, test

In [ ]:
def get_y_data():
    # output
    train_labels, val_labels, test_labels = (
        np.split(
            target_col, 
            [int(.6*len(target_col)), int(.8*len(target_col))])
    )
    
    return train_labels, val_labels, test_labels

##Escolher os 4 algoritmos para predição

Os algoritmos de predicão que iremos usar são:

*   Regressão Linear
*   Multilayer perceptron
*   Random forests
*   Gradient boost com lightgbm


In [ ]:
!pip install mlflow --quiet

     |████████████████████████████████| 14.4 MB 58 kB/s 
     |████████████████████████████████| 1.1 MB 27.1 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 170 kB 57.2 MB/s 
     |████████████████████████████████| 146 kB 39.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 636 kB 37.7 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [ ]:
!pip install optuna

     |████████████████████████████████| 302 kB 25.3 MB/s 
     |████████████████████████████████| 80 kB 7.5 MB/s 
     |████████████████████████████████| 111 kB 37.5 MB/s 
     |████████████████████████████████| 141 kB 56.5 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=ce5583930f9674fdda11cc41c652383fe2a3172f2f2bdc274bc583f8c3f3bea5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

import optuna

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import lightgbm
from lightgbm import LGBMRegressor

#Função de avaliação das metricas


In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    return rmse, mae

In [ ]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()
mlflow.lightgbm.autolog()

2021/08/16 22:49:00 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


###Regressão Linear

In [ ]:
def linear_regression(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run(run_name="Linear Regression"):
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predictions = reg.predict(val)

        (rmse, mae) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("Modelo de regressão linear")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        
        
        gc.collect()
        
        return rmse

In [ ]:
study = optuna.create_study()
study.optimize(linear_regression, n_trials=1)

[I 2021-08-16 23:40:05,292] A new study created in memory with name: no-name-5f57321a-8bf3-4278-b29f-00705065819f


Modelo de regressão linear
  RMSE: 0.019455898528420473
  MAE: 0.017646177370942265


[I 2021-08-16 23:40:05,869] Trial 0 finished with value: 0.019455898528420473 and parameters: {}. Best is trial 0 with value: 0.019455898528420473.


###Multilayer Perceptron

In [ ]:
def mlp(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    # hyper-parameters to test
    params = {
        "hidden_units": trial.suggest_int("hidden_units", 3, 15),
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Start an MLflow run
    with mlflow.start_run(run_name="MLP"):
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=1),
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=params["lr"]),
            loss='mean_squared_error'
        )

        history = mlp_model.fit(
            train, train_labels,
            validation_data=(test, test_labels),
            epochs=params["epochs"]
        )
        
        predictions = mlp_model.predict(val)

        (rmse, mae) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("MLP model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_name":"MultiLayerPerceptron",
                "estimator_class":"Keras"
            }
        )
        #mlflow.tensorflow.log_model(mlp_model, "model")
        #modelpath = "./mlflow/freight_value/model-mlp"
        #mlflow.tensorflow.save_model(mlp_model, modelpath)
        tf.keras.backend.clear_session()

        gc.collect()
        
        return rmse

In [ ]:

study = optuna.create_study()
study.optimize(mlp, n_trials=10)

[I 2021-08-16 23:40:08,086] A new study created in memory with name: no-name-339bd9e2-3bbd-45ce-ae8e-ba9ed0eee4a4


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 9)                 99        
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 10        
Total params: 310
Trainable params: 289
Non-trainable params: 21
_________________________________________________________________
Epoch 1/36
19/19 [==============================] - 1s 18ms/step - loss: 2.5022 - val_loss: 1.7990
Epoch 2/36
19/19 [======

[I 2021-08-16 23:40:15,216] Trial 0 finished with value: 0.09038554344523447 and parameters: {'hidden_units': 9, 'lr': 0.00022723168785531795, 'epochs': 36}. Best is trial 0 with value: 0.09038554344523447.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 3)                 33        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 82
Trainable params: 61
Non-trainable params: 21
_________________________________________________________________
Epoch 1/32
19/19 [==============================] - 1s 18ms/step - loss: 1.9708 - val_loss: 1.1704
Epoch 2/32
19/19 [========

[I 2021-08-16 23:40:21,849] Trial 1 finished with value: 0.4048236142076068 and parameters: {'hidden_units': 3, 'lr': 0.0002388773177567857, 'epochs': 32}. Best is trial 0 with value: 0.09038554344523447.


...

[I 2021-08-16 23:41:00,935] Trial 7 finished with value: 0.024214301133278424 and parameters: {'hidden_units': 15, 'lr': 0.0004983902441307072, 'epochs': 21}. Best is trial 3 with value: 0.021984397209475523.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 14)                154       
_________________________________________________________________
dense_1 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15        
Total params: 610
Trainable params: 589
Non-trainable params: 21
_________________________________________________________________
Epoch 1/21
19/19 [==============================] - 1s 17ms/step - loss: 0.9948 - val_loss: 0.3019
Epoch 2/21
19/19 [======

[I 2021-08-16 23:41:06,400] Trial 8 finished with value: 0.02244811951739125 and parameters: {'hidden_units': 14, 'lr': 0.000993719304677227, 'epochs': 21}. Best is trial 3 with value: 0.021984397209475523.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 14)                154       
_________________________________________________________________
dense_1 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15        
Total params: 610
Trainable params: 589
Non-trainable params: 21
_________________________________________________________________
Epoch 1/20
19/19 [==============================] - 1s 17ms/step - loss: 0.7459 - val_loss: 0.3865
Epoch 2/20
19/19 [======

[I 2021-08-16 23:41:13,838] Trial 9 finished with value: 0.05029729734074176 and parameters: {'hidden_units': 14, 'lr': 0.00024949420936076707, 'epochs': 20}. Best is trial 3 with value: 0.021984397209475523.


###Random Forest

In [ ]:
def random_forest(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Random Forest"):
        rf = RandomForestRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
            min_samples_split=params["min_samples_split"],
            random_state=0
        )
        rf.fit(train, train_labels)
        
        predictions = rf.predict(val)
        
        (rmse, mae) = eval_metrics(val_labels, predictions)
        
        print("Random Forest model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        
        gc.collect()
        
        return rmse

In [ ]:
study = optuna.create_study()
study.optimize(random_forest, n_trials=10)

[I 2021-08-16 23:43:53,234] A new study created in memory with name: no-name-8e7f2a4a-7367-45ce-9839-af0b857f7709


Random Forest model
  RMSE: 0.032229081407279016
  MAE: 0.02020424733275734


[I 2021-08-16 23:43:54,723] Trial 0 finished with value: 0.032229081407279016 and parameters: {'n_estimators': 135, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 0 with value: 0.032229081407279016.


Random Forest model
  RMSE: 0.029394081257748273
  MAE: 0.018991277859937185


[I 2021-08-16 23:43:55,872] Trial 1 finished with value: 0.029394081257748273 and parameters: {'n_estimators': 114, 'max_depth': 4, 'min_samples_split': 4}. Best is trial 1 with value: 0.029394081257748273.


...

[I 2021-08-16 23:44:03,553] Trial 7 finished with value: 0.03264915341859045 and parameters: {'n_estimators': 71, 'max_depth': 9, 'min_samples_split': 2}. Best is trial 4 with value: 0.026497436782335498.


Random Forest model
  RMSE: 0.03251603743389072
  MAE: 0.02010323987100584


[I 2021-08-16 23:44:04,603] Trial 8 finished with value: 0.03251603743389072 and parameters: {'n_estimators': 82, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 4 with value: 0.026497436782335498.


Random Forest model
  RMSE: 0.030847109810172392
  MAE: 0.018793560781238106


[I 2021-08-16 23:44:05,934] Trial 9 finished with value: 0.030847109810172392 and parameters: {'n_estimators': 124, 'max_depth': 6, 'min_samples_split': 3}. Best is trial 4 with value: 0.026497436782335498.


###Gradient boost com lightgbm

In [ ]:
def gradient_boosting(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "num_leaves": trial.suggest_int("num_leaves", 25, 35),
        "max_depth": trial.suggest_int("max_depth", 2, 10)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Gradient Boosting"):
#         model = LGBMRegressor(
#             max_depth=params["max_depth"],
#             n_estimators=params["n_estimators"],
#             num_leaves=params["num_leaves"],
#         )
        model = XGBRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
        )
        model.fit(train, train_labels)
        
        predictions = model.predict(test)
        print('Prediction: %.3f' % predictions[0])
        
        (rmse, mae) = eval_metrics(val_labels, predictions)

        print("LGBM model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_class":"LightGBM",
                "estimator_name":"Gradient Boosting"
            }
        )
        mlflow.sklearn.log_model(model, "model")
        
        gc.collect()
        
        return rmse

In [ ]:
study = optuna.create_study()
study.optimize(gradient_boosting, n_trials=10)

[I 2021-08-16 23:57:07,561] A new study created in memory with name: no-name-5f41e926-5ce7-4675-92d0-b73e68a73a51


[23:57:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.017
LGBM model
  RMSE: 0.03993479922439491
  MAE: 0.02109046509861946


[I 2021-08-16 23:57:08,128] Trial 0 finished with value: 0.03993479922439491 and parameters: {'n_estimators': 68, 'num_leaves': 34, 'max_depth': 6}. Best is trial 0 with value: 0.03993479922439491.


[23:57:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.006
LGBM model
  RMSE: 0.03989644219429306
  MAE: 0.022985045850276952


[I 2021-08-16 23:57:08,801] Trial 1 finished with value: 0.03989644219429306 and parameters: {'n_estimators': 129, 'num_leaves': 35, 'max_depth': 5}. Best is trial 1 with value: 0.03989644219429306.


...

[I 2021-08-16 23:57:13,040] Trial 7 finished with value: 0.039269274523501846 and parameters: {'n_estimators': 84, 'num_leaves': 31, 'max_depth': 8}. Best is trial 5 with value: 0.027320911839342425.


[23:57:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.023
LGBM model
  RMSE: 0.03886160671203333
  MAE: 0.02232185280323028


[I 2021-08-16 23:57:13,693] Trial 8 finished with value: 0.03886160671203333 and parameters: {'n_estimators': 51, 'num_leaves': 25, 'max_depth': 6}. Best is trial 5 with value: 0.027320911839342425.


[23:57:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.015
LGBM model
  RMSE: 0.04136983743460978
  MAE: 0.021324082970619197


[I 2021-08-16 23:57:14,517] Trial 9 finished with value: 0.04136983743460978 and parameters: {'n_estimators': 93, 'num_leaves': 35, 'max_depth': 6}. Best is trial 5 with value: 0.027320911839342425.
